In [23]:
library(lmerTest)
library(MuMIn)
library(VIF)
library(sqldf)

filename <- 'generated/metrics_for_time_series.csv'

TimeSeriesData <- read.csv(file=filename, header=TRUE, sep=",")

summary(TimeSeriesData)

TimeSeriesData <- sqldf("select *
                      from 'TimeSeriesData' 
                      where Period != 13")


TimeSeriesData$Intervention = TimeSeriesData$Period > 12
TimeSeriesData$TimeAfterIntervention = ifelse(TimeSeriesData$Period>12, TimeSeriesData$Period-12, 0)

summary(TimeSeriesData)

print(str(TimeSeriesData))

                         ProjectName         Language    ShareReviewComments
 activemerchant/active_merchant:  25   Ruby      :1200   Min.   :0.0000     
 adobe/brackets                :  25   Python    :1125   1st Qu.:0.0000     
 AFNetworking/AFNetworking     :  25   JavaScript: 925   Median :0.1186     
 airbnb/javascript             :  25   PHP       : 800   Mean   :0.1855     
 ajaxorg/ace                   :  25   C++       : 775   3rd Qu.:0.3090     
 alexreisner/geocoder          :  25   Java      : 700   Max.   :1.0000     
 (Other)                       :7450   (Other)   :2075                      
   Additions        Deletions       ChangedFiles      Assignees      
 Min.   : 0.000   Min.   :0.0000   Min.   :0.0000   Min.   :0.00000  
 1st Qu.: 1.586   1st Qu.:0.8815   1st Qu.:0.6931   1st Qu.:0.00000  
 Median : 2.382   Median :1.4915   Median :1.0104   Median :0.00000  
 Mean   : 2.423   Mean   :1.5884   Mean   :1.0583   Mean   :0.04135  
 3rd Qu.: 3.200   3rd Qu.:2.1859  

                         ProjectName         Language    ShareReviewComments
 activemerchant/active_merchant:  24   Ruby      :1152   Min.   :0.0000     
 adobe/brackets                :  24   Python    :1080   1st Qu.:0.0000     
 AFNetworking/AFNetworking     :  24   JavaScript: 888   Median :0.1176     
 airbnb/javascript             :  24   PHP       : 768   Mean   :0.1853     
 ajaxorg/ace                   :  24   C++       : 744   3rd Qu.:0.3085     
 alexreisner/geocoder          :  24   Java      : 672   Max.   :1.0000     
 (Other)                       :7152   (Other)   :1992                      
   Additions        Deletions       ChangedFiles      Assignees      
 Min.   : 0.000   Min.   :0.0000   Min.   :0.0000   Min.   :0.00000  
 1st Qu.: 1.580   1st Qu.:0.8791   1st Qu.:0.6931   1st Qu.:0.00000  
 Median : 2.377   Median :1.4868   Median :1.0071   Median :0.00000  
 Mean   : 2.421   Mean   :1.5869   Mean   :1.0568   Mean   :0.04136  
 3rd Qu.: 3.199   3rd Qu.:2.1857  

'data.frame':	7296 obs. of  17 variables:
 $ ProjectName          : Factor w/ 304 levels "activemerchant/active_merchant",..: 128 128 128 128 128 128 128 128 128 128 ...
 $ Language             : Factor w/ 36 levels "","Assembly",..: 16 16 16 16 16 16 16 16 16 16 ...
 $ ShareReviewComments  : num  0.022 0.0508 0 0.0303 0.0385 ...
 $ Additions            : num  2.472 1.575 0.449 1.364 3.142 ...
 $ Deletions            : num  0.822 0.515 0.063 0.69 1.401 ...
 $ ChangedFiles         : num  0.892 0.524 0.189 0.565 1 ...
 $ Assignees            : num  0 0 0.063 0.0365 0.1066 ...
 $ ReviewComments       : num  0.0354 0.0478 0 0.0578 0.0845 ...
 $ GeneralComments      : num  1.021 0.801 1.186 1.045 1.213 ...
 $ TotalPrs             : int  31 29 11 19 13 43 35 23 35 29 ...
 $ Commits              : num  1.42 1.06 1.05 1.04 1.21 ...
 $ CommitsAfterCreate   : num  0.349 0.408 0.389 0.335 0.399 ...
 $ EffectiveComments    : num  0 0 0 0 0 ...
 $ NumOfUniqueUsers     : num  0.859 0.679 1.011 0.821

In [18]:
vif.mer <- function (fit) {
    ## adapted from rms::vif
    
    v <- vcov(fit)
    nam <- names(fixef(fit))

    ## exclude intercepts
    ns <- sum(1 * (nam == "Intercept" | nam == "(Intercept)"))
    if (ns > 0) {
        v <- v[-(1:ns), -(1:ns), drop = FALSE]
        nam <- nam[-(1:ns)]
    }
    
    d <- diag(v)^0.5
    v <- diag(solve(v/(d %o% d)))
    names(v) <- nam
    v
}

print(TimeSeriesData$Language[1])
print(levels(TimeSeriesData$Language))[3]

[1] JavaScript
36 Levels:  Assembly C C# C++ CartoCSS Clojure CSS DM Emacs Lisp Erlang ... TypeScript
 [1] ""            "Assembly"    "C"           "C#"          "C++"        
 [6] "CartoCSS"    "Clojure"     "CSS"         "DM"          "Emacs Lisp" 
[11] "Erlang"      "Go"          "Haskell"     "HTML"        "Java"       
[16] "JavaScript"  "Lua"         "Makefile"    "Nim"         "Objective-C"
[21] "OCaml"       "Perl"        "Perl 6"      "PHP"         "PLpgSQL"    
[26] "Puppet"      "Python"      "R"           "Racket"      "Ruby"       
[31] "Rust"        "Scala"       "Shell"       "Swift"       "TeX"        
[36] "TypeScript" 


[1] "C"

In [3]:
projects <- sqldf("select count(distinct(ProjectName)) from TimeSeriesData")

print(projects)

  count(distinct(ProjectName))
1                          304


In [34]:
modelNumberReviewComments = lmer(ReviewComments ~ 
            Additions+
            Deletions  +
            Commits  +
            CommitsAfterCreate  +
            Assignees  + 
            ChangedFiles + 
            TotalPrs +
            Intervention +
            Period + 
            TimeAfterIntervention +
            (1|Language) +
            (1+Intervention|ProjectName),
          data= TimeSeriesData, 
          REML=FALSE)
summary(modelNumberReviewComments)
r.squaredGLMM(modelNumberReviewComments)
vif.mer(modelNumberReviewComments)
anova(modelNumberReviewComments)

random_row <- data.frame(
    Additions = c(median(TimeSeriesData$Additions),median(TimeSeriesData$Additions)),
    Deletions = c(median(TimeSeriesData$Deletions),median(TimeSeriesData$Deletions)),
    Commits = c(median(TimeSeriesData$Commits),median(TimeSeriesData$Commits)),
    CommitsAfterCreate = c(median(TimeSeriesData$CommitsAfterCreate), median(TimeSeriesData$CommitsAfterCreate)),
    Assignees = c(median(TimeSeriesData$Assignees), median(TimeSeriesData$Assignees)),
    ChangedFiles = c(median(TimeSeriesData$ChangedFiles), median(TimeSeriesData$ChangedFiles)),
    TotalPrs = c(median(TimeSeriesData$TotalPrs), median(TimeSeriesData$TotalPrs)),
    Intervention = c(FALSE, TRUE),
    Period = c(24,0),
    TimeAfterIntervention = c(0,12),
    Language = c(levels(TimeSeriesData$Language)[4], levels(TimeSeriesData$Language)[4]),
    ProjectName = c(levels(TimeSeriesData$ProjectName)[4], levels(TimeSeriesData$ProjectName)[4])
)
print(predict(modelNumberReviewComments, random_row))

Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: 
ReviewComments ~ Additions + Deletions + Commits + CommitsAfterCreate +  
    Assignees + ChangedFiles + TotalPrs + Intervention + Period +  
    TimeAfterIntervention + (1 | Language) + (1 + Intervention |  
    ProjectName)
   Data: TimeSeriesData

     AIC      BIC   logLik deviance df.resid 
 -2129.4  -2019.1   1080.7  -2161.4     7280 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-7.8771 -0.4550 -0.0613  0.3495 12.5598 

Random effects:
 Groups      Name             Variance Std.Dev. Corr 
 ProjectName (Intercept)      0.027985 0.1673        
             InterventionTRUE 0.015201 0.1233   -0.10
 Language    (Intercept)      0.001037 0.0322        
 Residual                     0.036465 0.1910        
Number of obs: 7296, groups:  ProjectName, 304; Language, 36

Fixed effects:
                        Estimate Std. Error         df t valu

R2m       R2c 
0.1650380 0.5715845

Additions             Deletions               Commits 
             3.901234              2.742011              2.030683 
   CommitsAfterCreate             Assignees          ChangedFiles 
             1.725214              1.012810              4.446489 
             TotalPrs      InterventionTRUE                Period 
             1.019980              2.039765              4.079702 
TimeAfterIntervention 
             3.360853

,Sum Sq,Mean Sq,NumDF,DenDF,F.value,Pr(>F)
Additions,3.05602445,3.05602445,1,7147.685,83.8073224,0.000000e+00
Deletions,0.03067253,0.03067253,1,7099.627,0.8411525,3.590985e-01
Commits,0.46029241,0.46029241,1,7128.794,12.6228944,3.835146e-04
CommitsAfterCreate,26.97346733,26.97346733,1,7154.033,739.7107294,0.000000e+00
Assignees,0.02742935,0.02742935,1,6242.464,0.7522127,3.858099e-01
ChangedFiles,0.40706138,0.40706138,1,7192.792,11.1631058,8.386528e-04
TotalPrs,0.30247012,0.30247012,1,5526.145,8.2948324,3.991092e-03
Intervention,0.04523005,0.04523005,1,1203.349,1.2403726,2.656217e-01
Period,1.43748963,1.43748963,1,6706.454,39.4212021,3.629086e-10
TimeAfterIntervention,0.35065926,0.35065926,1,6685.717,9.6163541,1.936549e-03


        1         2 
0.3970056 0.2258874 


In [36]:
modelShareReviewComments = lmer(GeneralComments  ~ 
            Additions  +
            Deletions + 
            Commits + 
            Assignees +
            ChangedFiles +
            TotalPrs +
            CommitsAfterCreate +
            Intervention +
            Period + 
            TimeAfterIntervention +
            (1|Language) +
            (1+Intervention|ProjectName),
          data= TimeSeriesData, 
          REML=FALSE)
summary(modelShareReviewComments)
r.squaredGLMM(modelShareReviewComments)
vif.mer(modelShareReviewComments)
anova(modelShareReviewComments)

random_row <- data.frame(
    Additions = c(median(TimeSeriesData$Additions),median(TimeSeriesData$Additions)),
    Deletions = c(median(TimeSeriesData$Deletions),median(TimeSeriesData$Deletions)),
    Commits = c(median(TimeSeriesData$Commits),median(TimeSeriesData$Commits)),
    CommitsAfterCreate = c(median(TimeSeriesData$CommitsAfterCreate), median(TimeSeriesData$CommitsAfterCreate)),
    Assignees = c(median(TimeSeriesData$Assignees), median(TimeSeriesData$Assignees)),
    ChangedFiles = c(median(TimeSeriesData$ChangedFiles), median(TimeSeriesData$ChangedFiles)),
    TotalPrs = c(median(TimeSeriesData$TotalPrs), median(TimeSeriesData$TotalPrs)),
    Intervention = c(FALSE, TRUE),
    Period = c(24,0),
    TimeAfterIntervention = c(0,12),
    Language = c(levels(TimeSeriesData$Language)[4], levels(TimeSeriesData$Language)[4]),
    ProjectName = c(levels(TimeSeriesData$ProjectName)[4], levels(TimeSeriesData$ProjectName)[4])
)
print(predict(modelShareReviewComments, random_row))

Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: GeneralComments ~ Additions + Deletions + Commits + Assignees +  
    ChangedFiles + TotalPrs + CommitsAfterCreate + Intervention +  
    Period + TimeAfterIntervention + (1 | Language) + (1 + Intervention |  
    ProjectName)
   Data: TimeSeriesData

     AIC      BIC   logLik deviance df.resid 
  3764.4   3874.7  -1866.2   3732.4     7280 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-7.2595 -0.5367 -0.0113  0.5112  6.5815 

Random effects:
 Groups      Name             Variance Std.Dev. Corr 
 ProjectName (Intercept)      0.09570  0.3094        
             InterventionTRUE 0.04931  0.2221   -0.23
 Language    (Intercept)      0.00000  0.0000        
 Residual                     0.07963  0.2822        
Number of obs: 7296, groups:  ProjectName, 304; Language, 36

Fixed effects:
                        Estimate Std. Error         df t valu

R2m       R2c 
0.1118741 0.6157333

Additions             Deletions               Commits 
             3.867863              2.711852              2.029071 
            Assignees          ChangedFiles              TotalPrs 
             1.012430              4.407883              1.018104 
   CommitsAfterCreate      InterventionTRUE                Period 
             1.728237              1.772122              3.553267 
TimeAfterIntervention 
             3.012843

,Sum Sq,Mean Sq,NumDF,DenDF,F.value,Pr(>F)
Additions,2.7262871,2.7262871,1,7057.7942,34.236416,5.099109e-09
Deletions,1.2803726,1.2803726,1,7021.7976,16.078779,6.139084e-05
Commits,1.2254352,1.2254352,1,7127.2826,15.388882,8.832362e-05
Assignees,2.7053580,2.7053580,1,6842.3734,33.973591,5.840612e-09
ChangedFiles,0.3906977,0.3906977,1,7109.7922,4.906339,2.678987e-02
TotalPrs,1.1559940,1.1559940,1,6299.0639,14.516847,1.402344e-04
CommitsAfterCreate,47.6993481,47.6993481,1,7086.7785,599.003215,0.000000e+00
Intervention,0.7565021,0.7565021,1,922.4574,9.500071,2.115677e-03
Period,2.6002375,2.6002375,1,6706.5633,32.653499,1.148673e-08
TimeAfterIntervention,1.0711651,1.0711651,1,6689.8780,13.451574,2.467014e-04


        1         2 
0.9960805 0.6150564 


In [6]:
modelShareReviewComments = lmer(EffectiveComments ~ 
            Additions + 
            Deletions + 
            Commits + 
            Assignees + 
            ChangedFiles + 
            #GeneralComments + 
            TotalPrs +
            Intervention +
            Period + 
            TimeAfterIntervention +
            (1|Language) +
            (1+Intervention|ProjectName),
          data= TimeSeriesData, 
          REML=FALSE)
summary(modelShareReviewComments)
r.squaredGLMM(modelShareReviewComments)
vif.mer(modelShareReviewComments)
anova(modelShareReviewComments)

Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: EffectiveComments ~ Additions + Deletions + Commits + Assignees +  
    ChangedFiles + TotalPrs + Intervention + Period + TimeAfterIntervention +  
    (1 | Language) + (1 + Intervention | ProjectName)
   Data: TimeSeriesData

     AIC      BIC   logLik deviance df.resid 
-10200.7 -10097.3   5115.4 -10230.7     7281 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.9855 -0.3924 -0.0927  0.2136 21.5281 

Random effects:
 Groups      Name             Variance  Std.Dev. Corr 
 ProjectName (Intercept)      0.0053491 0.07314       
             InterventionTRUE 0.0032844 0.05731  -0.29
 Language    (Intercept)      0.0002394 0.01547       
 Residual                     0.0125468 0.11201       
Number of obs: 7296, groups:  ProjectName, 304; Language, 36

Fixed effects:
                        Estimate Std. Error         df t value Pr(>|t|)    
(Inte

R2m       R2c 
0.1049375 0.3948804

Additions             Deletions               Commits 
             3.947200              2.814461              1.319891 
            Assignees          ChangedFiles              TotalPrs 
             1.008965              4.490576              1.021025 
     InterventionTRUE                Period TimeAfterIntervention 
             2.415153              4.806177              3.845778

,Sum Sq,Mean Sq,NumDF,DenDF,F.value,Pr(>F)
Additions,1.296957960,1.296957960,1,7275.612,103.3698477,0.000000e+00
Deletions,0.363749784,0.363749784,1,7223.170,28.9915024,7.498454e-08
Commits,5.310722393,5.310722393,1,7250.522,423.2739855,0.000000e+00
Assignees,0.010527377,0.010527377,1,4204.098,0.8390506,3.597210e-01
ChangedFiles,0.239565637,0.239565637,1,7269.723,19.0938058,1.261778e-05
TotalPrs,0.005036171,0.005036171,1,3616.386,0.4013917,5.264114e-01
Intervention,0.006585597,0.006585597,1,1622.521,0.5248837,4.688703e-01
Period,0.391250747,0.391250747,1,6711.094,31.1833778,2.439582e-08
TimeAfterIntervention,0.267699412,0.267699412,1,6686.646,21.3361175,3.926174e-06


In [7]:
modelShareReviewComments = lmer(CommitsAfterCreate ~ 
            Additions + 
            Deletions + 
            Commits + 
            Assignees + 
            ChangedFiles + 
            TotalPrs +
            GeneralComments + 
            Intervention +
            Period + 
            TimeAfterIntervention +
            (1|Language) +
            (1+Intervention|ProjectName),
          data= TimeSeriesData, 
          REML=FALSE)
summary(modelShareReviewComments)
r.squaredGLMM(modelShareReviewComments)
vif.mer(modelShareReviewComments)
anova(modelShareReviewComments)

Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: CommitsAfterCreate ~ Additions + Deletions + Commits + Assignees +  
    ChangedFiles + TotalPrs + GeneralComments + Intervention +  
    Period + TimeAfterIntervention + (1 | Language) + (1 + Intervention |  
    ProjectName)
   Data: TimeSeriesData

     AIC      BIC   logLik deviance df.resid 
 -4958.7  -4848.3   2495.3  -4990.7     7280 

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-10.8614  -0.4345  -0.0085   0.4173  11.3471 

Random effects:
 Groups      Name             Variance Std.Dev. Corr 
 ProjectName (Intercept)      0.009398 0.09694       
             InterventionTRUE 0.006582 0.08113  -0.62
 Language    (Intercept)      0.000640 0.02530       
 Residual                     0.026253 0.16203       
Number of obs: 7296, groups:  ProjectName, 304; Language, 36

Fixed effects:
                        Estimate Std. Error        

R2m       R2c 
0.5475904 0.6580968

Additions             Deletions               Commits 
             4.041058              2.871336              1.348572 
            Assignees          ChangedFiles              TotalPrs 
             1.013819              4.547708              1.020562 
      GeneralComments      InterventionTRUE                Period 
             1.063692              2.456434              4.900280 
TimeAfterIntervention 
             3.901847

,Sum Sq,Mean Sq,NumDF,DenDF,F.value,Pr(>F)
Additions,1.166546e+00,1.166546e+00,1,7230.328,4.443454e+01,2.823652e-11
Deletions,4.543434e-02,4.543434e-02,1,7249.877,1.730626e+00,1.883726e-01
Commits,1.054670e+02,1.054670e+02,1,6999.949,4.017311e+03,0.000000e+00
Assignees,2.730551e-01,2.730551e-01,1,2443.285,1.040086e+01,1.276184e-03
ChangedFiles,1.348665e+00,1.348665e+00,1,7130.359,5.137158e+01,8.411050e-13
TotalPrs,3.365115e-02,3.365115e-02,1,2125.484,1.281796e+00,2.576931e-01
GeneralComments,1.690303e+01,1.690303e+01,1,5072.985,6.438482e+02,0.000000e+00
Intervention,2.374365e-03,2.374365e-03,1,1645.993,9.044123e-02,7.636550e-01
Period,1.556662e-01,1.556662e-01,1,6735.611,5.929434e+00,1.491581e-02
TimeAfterIntervention,6.721950e-02,6.721950e-02,1,6699.351,2.560438e+00,1.096154e-01


# how about if we don't apply any aggregation

In [29]:
filename_no_aggr <- 'generated/metrics_for_time_series_no_aggr.csv'

TimeSeriesDataNoAggr <- read.csv(file=filename_no_aggr, header=TRUE, sep=",")

summary(TimeSeriesDataNoAggr)

TimeSeriesDataNoAggr <- sqldf("select *
                      from 'TimeSeriesDataNoAggr' 
                      where Period != 13")


TimeSeriesDataNoAggr$Intervention = TimeSeriesDataNoAggr$Period > 12
TimeSeriesDataNoAggr$TimeAfterIntervention = ifelse(TimeSeriesDataNoAggr$Period>12, TimeSeriesDataNoAggr$Period-12, 0)

                       ProjectName       Period      EffectiveComments
 adobe/brackets              : 638   Min.   : 1.00   Min.   :  1.000  
 caskdata/cdap               : 633   1st Qu.: 7.00   1st Qu.:  1.000  
 DynamoDS/Dynamo             : 420   Median :13.00   Median :  2.000  
 symfony/symfony-docs        : 370   Mean   :12.93   Mean   :  6.055  
 AnalyticalGraphicsInc/cesium: 261   3rd Qu.:19.00   3rd Qu.:  5.000  
 cakephp/docs                : 253   Max.   :25.00   Max.   :190.000  
 (Other)                     :1966                                    
 ReviewThreads    GeneralComments     Additions          Deletions       
 Min.   :  1.00   Min.   :  0.000   Min.   :     0.0   Min.   :     0.0  
 1st Qu.:  2.00   1st Qu.:  2.000   1st Qu.:    13.0   1st Qu.:     1.0  
 Median :  5.00   Median :  4.000   Median :    87.0   Median :    12.0  
 Mean   : 12.55   Mean   :  8.498   Mean   :   819.5   Mean   :   322.8  
 3rd Qu.: 13.00   3rd Qu.:  9.000   3rd Qu.:   347.0   3rd Qu.

In [30]:
projects <- sqldf("select count(distinct(ProjectName)) from TimeSeriesDataNoAggr")

print(projects)

  count(distinct(ProjectName))
1                           18


In [31]:
modelEffectiveCommentsNoAggr = lmer(log(EffectiveComments + 1) ~ 
            log(Additions + 1) +
            log(Deletions + 1) +
            log(Commits + 1) +
            log(Assignees + 1) + 
            log(ChangedFiles + 1) + 
            IsMerged + 
            Intervention +
            Period + 
            TimeAfterIntervention +
            (1+Intervention|ProjectName),
          data= TimeSeriesDataNoAggr, 
          REML=FALSE)
summary(modelEffectiveCommentsNoAggr)
r.squaredGLMM(modelEffectiveCommentsNoAggr)
vif.mer(modelEffectiveCommentsNoAggr)
anova(modelEffectiveCommentsNoAggr)

Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: log(EffectiveComments + 1) ~ log(Additions + 1) + log(Deletions +  
    1) + log(Commits + 1) + log(Assignees + 1) + log(ChangedFiles +  
    1) + IsMerged + Intervention + Period + TimeAfterIntervention +  
    (1 + Intervention | ProjectName)
   Data: TimeSeriesDataNoAggr

     AIC      BIC   logLik deviance df.resid 
  9171.4   9260.6  -4571.7   9143.4     4317 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.5157 -0.6843 -0.1494  0.5464  4.4667 

Random effects:
 Groups      Name             Variance Std.Dev. Corr
 ProjectName (Intercept)      0.02523  0.1588       
             InterventionTRUE 0.01784  0.1336   0.00
 Residual                     0.47627  0.6901       
Number of obs: 4331, groups:  ProjectName, 18

Fixed effects:
                        Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)            3.615e-01  

R2m       R2c 
0.2533677 0.3031359

log(Additions + 1)    log(Deletions + 1)      log(Commits + 1) 
             3.185133              2.281099              1.398733 
   log(Assignees + 1) log(ChangedFiles + 1)          IsMergedTrue 
             1.009230              3.720643              1.036548 
     InterventionTRUE                Period TimeAfterIntervention 
             2.047772              3.804132              3.322282

,Sum Sq,Mean Sq,NumDF,DenDF,F.value,Pr(>F)
log(Additions + 1),75.04443963,75.04443963,1,4321.977,157.56617240,0.00000000
log(Deletions + 1),57.90042174,57.90042174,1,4317.630,121.56993748,0.00000000
log(Commits + 1),349.96244096,349.96244096,1,4319.894,734.79451083,0.00000000
log(Assignees + 1),4.23595771,4.23595771,1,3574.080,8.89397863,0.00288045
log(ChangedFiles + 1),2.82422692,2.82422692,1,4328.159,5.92985474,0.01492659
IsMerged,0.15024479,0.15024479,1,4303.772,0.31545971,0.57437830
Intervention,0.02744097,0.02744097,1,60.608,0.05761611,0.81111424
Period,2.46823576,2.46823576,1,4261.915,5.18240209,0.02286613
TimeAfterIntervention,2.09320811,2.09320811,1,4274.361,4.39497972,0.03610353
